In [1]:
import os
import json
import pandas as pd
from pathlib import Path

In [2]:
result_path = Path(os.getcwd()) / "results"

In [3]:
result_summary = {}
summary_file_name = "summary.json"

In [4]:
for llm in os.listdir(result_path):
    if os.path.isdir(result_path / llm):
        result_summary[llm] = {}
        for task in os.listdir(result_path / llm):
            if os.path.isdir(result_path / llm / task):
                result_summary[llm][task] = {}
                for pe in os.listdir(result_path / llm / task):
                    if os.path.isdir(result_path / llm / task / pe):
                        result_summary[llm][task][pe] = {}
                        files = [file for file in os.listdir(result_path / llm / task / pe) if
                                 file != summary_file_name and file != ".DS_Store"]
                        with open(result_path / llm / task / pe / summary_file_name, "r") as f:
                            result_summary[llm][task][pe] = json.load(f)

In [5]:
result_summary

{'gemini-pro-chat': {'anli': {'zero-shot': {'model': 'gemini-pro-chat',
    'task': 'anli',
    'pe_technique': 'zero-shot',
    'current_index': 1200,
    'is_processing': False,
    'correct': 613,
    'total': 1200,
    'accuracy': 51.083333333333336,
    'created_at': '2024-02-11-20-55-25'},
   'null-shot-cot': {'model': 'gemini-pro-chat',
    'task': 'anli',
    'pe_technique': 'null-shot-cot',
    'current_index': 1200,
    'is_processing': False,
    'correct': 540,
    'total': 1200,
    'accuracy': 45.0,
    'created_at': '2024-02-11-20-55-27'},
   'zero-shot-cot': {'model': 'gemini-pro-chat',
    'task': 'anli',
    'pe_technique': 'zero-shot-cot',
    'current_index': 1200,
    'is_processing': False,
    'correct': 587,
    'total': 1200,
    'accuracy': 48.91666666666667,
    'created_at': '2024-02-11-20-55-26'},
   'null-shot': {'model': 'gemini-pro-chat',
    'task': 'anli',
    'pe_technique': 'null-shot',
    'current_index': 1200,
    'is_processing': False,
    'corr

In [11]:
for model in result_summary.keys():
    for task in result_summary[model].keys():
        for pe in result_summary[model][task].keys():
            lengths = []
            time_takens = []
            files = [file for file in os.listdir(result_path / model / task / pe) if file != summary_file_name and file != ".DS_Store"]
            for file in files:
                with open(Path(result_path) / model / task / pe / file) as f:
                    data = json.load(f)
                    lengths.append(data["length"])
                    time_takens.append(data["time_taken"])
            mean_length = sum(lengths) / len(lengths)
            mean_time_taken = sum(time_takens) / len(time_takens)
            std_length = (sum([(length - mean_length) ** 2 for length in lengths]) / len(lengths)) ** 0.5
            std_time_taken = (sum([(time_taken - mean_time_taken) ** 2 for time_taken in time_takens]) / len(time_takens)) ** 0.5
            min_length = min(lengths)
            min_time_taken = min(time_takens)
            max_length = max(lengths)
            max_time_taken = max(time_takens)
            median_length = sorted(lengths)[len(lengths) // 2]
            median_time_taken = sorted(time_takens)[len(time_takens) // 2]
            total_time_taken = sum(time_takens)
            result_summary[model][task][pe]["mean_length"] = mean_length
            result_summary[model][task][pe]["median_length"] = median_length
            result_summary[model][task][pe]["std_length"] = std_length
            result_summary[model][task][pe]["min_length"] = min_length
            result_summary[model][task][pe]["max_length"] = max_length
            result_summary[model][task][pe]["mean_time_taken"] = mean_time_taken
            result_summary[model][task][pe]["median_time_taken"] = median_time_taken
            result_summary[model][task][pe]["std_time_taken"] = std_time_taken
            result_summary[model][task][pe]["min_time_taken"] = min_time_taken
            result_summary[model][task][pe]["max_time_taken"] = max_time_taken
            result_summary[model][task][pe]["total_time_taken"] = total_time_taken            

In [7]:
df = pd.DataFrame.from_dict({(i, j, k): result_summary[i][j][k]
                             for i in result_summary.keys()
                             for j in result_summary[i].keys()
                             for k in result_summary[i][j].keys()},
                            orient='index')

In [8]:
df = df.drop(columns=["current_index", "is_processing"])

In [9]:
df

model      task  \
gemini-pro-chat anli     zero-shot        gemini-pro-chat      anli   
                         null-shot-cot    gemini-pro-chat      anli   
                         zero-shot-cot    gemini-pro-chat      anli   
                         null-shot        gemini-pro-chat      anli   
                triviaqa zero-shot        gemini-pro-chat  triviaqa   
...                                                   ...       ...   
gpt-3.5-turbo   race-h   null-shot-after    gpt-3.5-turbo    race-h   
                         null-shot-v2       gpt-3.5-turbo    race-h   
                         zero-shot-cot      gpt-3.5-turbo    race-h   
                         null-shot          gpt-3.5-turbo    race-h   
                         null-shot-v3       gpt-3.5-turbo    race-h   

                                             pe_technique  correct  total  \
gemini-pro-chat anli     zero-shot              zero-shot      613   1200   
                         null-shot-cot      null-shot-cot      540   1200   
                         zero-shot-cot      zero-shot-cot      587   1200   
                         null-shot              null-shot      623   1200   
                triviaqa zero-shot              zero-shot      705   1000   
...                                                   ...      ...    ...   
gpt-3.5-turbo   race-h   null-shot-after  null-shot-after     2727   3498   
                         null-shot-v2        null-shot-v2     2836   3498   
                         zero-shot-cot      zero-shot-cot     2717   3498   
                         null-shot              null-shot     2825   3498   
                         null-shot-v3        null-shot-v3     2818   3498   

                                           accuracy           created_at  \
gemini-pro-chat anli     zero-shot        51.083333  2024-02-11-20-55-25   
                         null-shot-cot    45.000000  2024-02-11-20-55-27   
                         zero-shot-cot    48.916667  2024-02-11-20-55-26   
                         null-shot        51.916667  2024-02-11-20-55-25   
                triviaqa zero-shot        70.500000  2024-02-11-20-55-34   
...                                             ...                  ...   
gpt-3.5-turbo   race-h   null-shot-after  77.958834  2024-01-03-13-17-11   
                         null-shot-v2     81.074900  2024-01-03-13-17-17   
                         zero-shot-cot    77.672956  2024-01-03-13-18-42   
                         null-shot        80.760435  2024-01-03-13-18-38   
                         null-shot-v3     80.560320  2024-01-03-13-17-20   

                                          mean_length  median_length  \
gemini-pro-chat anli     zero-shot          13.236667             13   
                         null-shot-cot      80.192500             16   
                         zero-shot-cot      14.296667             16   
                         null-shot          14.009167             13   
                triviaqa zero-shot          13.327000              9   
...                                               ...            ...   
gpt-3.5-turbo   race-h   null-shot-after   232.633505            223   
                         null-shot-v2       40.795597             39   
                         zero-shot-cot     419.522584            367   
                         null-shot          83.260720             44   
                         null-shot-v3       40.559748             39   

                                          std_length  min_length  max_length  \
gemini-pro-chat anli     zero-shot          3.081773           0          16   
                         null-shot-cot    143.966560           0         875   
                         zero-shot-cot      2.788725           0          16   
                         null-shot          2.414004           0          16   
                triviaqa zero-shot         61.193595           0        1779   
...             

In [10]:
df.to_csv("summary.csv")